<a href="https://colab.research.google.com/github/jsstar522/hunkim_ML/blob/master/03_NeuralNet_ReLU/01_ReLU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ReLU의 등장

## Sigmoid의 한계

Logistic Regression에서 출력값을 구하는 함수(Hypothesis, Activation Function)로 Sigmoid를 사용했습니다. 0에서 1이 나오는 함수입니다. 하지만 많은 hidden layer에 이 sigmoid 함수를 적용시킬 때 문제가 발생했습니다.

In [1]:
import numpy as np
import tensorflow as tf

x_data = np.array([[0, 0], [0, 1], [1, 0], [1, 1]], dtype=np.float32)
y_data = np.array([[0], [1], [1], [0]], dtype=np.float32)

X = tf.placeholder(tf.float32)
Y = tf.placeholder(tf.float32)

W1 = tf.Variable(tf.random_normal([2,5]), name="wieght1")
W2 = tf.Variable(tf.random_normal([5,5]), name="wieght2")
W3 = tf.Variable(tf.random_normal([5,5]), name="wieght3")
W4 = tf.Variable(tf.random_normal([5,5]), name="wieght4")
W5 = tf.Variable(tf.random_normal([5,5]), name="wieght5")
W6 = tf.Variable(tf.random_normal([5,5]), name="wieght6")
W7 = tf.Variable(tf.random_normal([5,5]), name="wieght7")
W8 = tf.Variable(tf.random_normal([5,5]), name="wieght8")
W9 = tf.Variable(tf.random_normal([5,5]), name="wieght9")
W10 = tf.Variable(tf.random_normal([5,5]), name="wieght10")
W11 = tf.Variable(tf.random_normal([5,1]), name="wieght11")

b1 = tf.Variable(tf.random_normal([5]), name="bias1")
b2 = tf.Variable(tf.random_normal([5]), name="bias2")
b3 = tf.Variable(tf.random_normal([5]), name="bias3")
b4 = tf.Variable(tf.random_normal([5]), name="bias4")
b5 = tf.Variable(tf.random_normal([5]), name="bias5")
b6 = tf.Variable(tf.random_normal([5]), name="bias6")
b7 = tf.Variable(tf.random_normal([5]), name="bias7")
b8 = tf.Variable(tf.random_normal([5]), name="bias8")
b9 = tf.Variable(tf.random_normal([5]), name="bias9")
b10 = tf.Variable(tf.random_normal([5]), name="bias10")
b11 = tf.Variable(tf.random_normal([1]), name="bias11")


L1 = tf.sigmoid(tf.matmul(X, W1) + b1)
L2 = tf.sigmoid(tf.matmul(L1, W2) + b2)
L3 = tf.sigmoid(tf.matmul(L2, W3) + b3)
L4 = tf.sigmoid(tf.matmul(L3, W4) + b4)
L5 = tf.sigmoid(tf.matmul(L4, W5) + b5)
L6 = tf.sigmoid(tf.matmul(L5, W6) + b6)
L7 = tf.sigmoid(tf.matmul(L6, W7) + b7)
L8 = tf.sigmoid(tf.matmul(L7, W8) + b8)
L9 = tf.sigmoid(tf.matmul(L8, W9) + b9)
L10 = tf.sigmoid(tf.matmul(L9, W10) + b10)

hypothesis = tf.sigmoid(tf.matmul(L10, W11) + b11)


cost = -tf.reduce_mean(Y * tf.log(hypothesis) + (1 - Y) * tf.log(1 - hypothesis))
train = tf.train.GradientDescentOptimizer(learning_rate=0.1).minimize(cost)

predicted = tf.cast(hypothesis > 0.5, dtype=tf.float32)
accuracy = tf.reduce_mean(tf.cast(tf.equal(predicted, Y), dtype=tf.float32))

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    for step in range(10001):
        sess.run(train, feed_dict={X: x_data, Y: y_data})
#         if step % 100 == 0:
#           print(step, sess.run(cost, feed_dict={X: x_data, Y: y_data}), sess.run(W2))
            
    h, c, a = sess.run([hypothesis, predicted, accuracy], feed_dict={X: x_data, Y: y_data})
    print("\nHypothesis: ", h, "\nCorrect: ", c, "\nAccuracy: ", a)



Hypothesis:  [[0.5000012]
 [0.5000011]
 [0.5000011]
 [0.5000008]] 
Correct:  [[1.]
 [1.]
 [1.]
 [1.]] 
Accuracy:  0.5


위와 같이 10개의 hidden layer를 사용했음에도 불구하고 정확도가 50% 밖에 나오지 않습니다. 정작 2개의 hidden layer를 사용할 때보다 더 낮은 성능을 보여줍니다. 이는 많은 layer가 있을 때, **Vanishing Gradient**가 생기기 때문입니다. Vanishing Gradient란 수많은 hidden layer가 있을 때, backpropagation을 통해 최초 X가 최종 Y에 얼마나 영향을 미치는지 **편미분을 통해서 거슬러 올라가는 과정에서 기울기를 잃어버리는 것입니다.**

$$\frac{af}{ag}\frac{ag}{ax}$$

편미분은 결국 두 미분값의 **곱**으로 나타나게 됩니다. 뒤에 있는 $$\frac{ag}{ax}$$ 은 하나의 Layer를 통과한 결과값입니다. 
![1.jepg](1.jpeg)[그림1]*최초 W가 최종 Cost에 미치는 영향을 Chain Rule로 푸는 과정*

[그림1]을 보면 chain rule 사이사이에 있는 항들이 weight값과 sigmoid 미분값의 곱으로 표현되어 있는 것을 알 수 있습니다. **weight 값은 평균 0, std가 1인 정규분포 안의 숫자**이고 **sigmoid 함수는 미분해보면 최소 0, 최대 1/4 값**이 나오는 것을 알 수 있습니다. [그림1]에서는 hidden layer가 2개밖에 되지 않아 vanishing gradient 현상이 나타나진 않지만 layer가 많아지면 많아질수록 (deep해질수록) **0~1 숫자가 계속 곱해지므로 결국 최초 weight가 error에 미치는 영향이 0에 수렴**하는 결과가 나옵니다. 

Weight는 다음과 같이 Optimize 됩니다.

$$W: W - a\frac{\alpha cost}{\alpha W}$$

위에서 vanishing gradient에 의해 W가 error(cost)에 미치는 영향이 0에 수렴하므로 weight는 좋은성능으로 학습하는 방향으로 움직이지 않습니다.

(참고 https://brunch.co.kr/@chris-song/39)

## ReLu

Relu는 0~1의 범위로 값을 추출하는 sigmoid의 한계를 다음과 같이 해결합니다.
![2.png](2.png)*[그림2] Sigmoid의 한계를 ReLU로 해결*
ReLU는 0이하의 값은 모두 0으로 처리하고 0이상의 값들은 모두 그대로 사용합니다. 이런 값들을 **sparse (넓게 퍼져있는, dense의 반대)**하다고 합니다. sparse한 값은 값이 변하는 데에 있어 더 좋은 성능을 보입니다. hidden layer에서 나온 output들은 모두 ReLU로 처리하고 마지막 global output만 sigmoid로 처리합니다.


In [21]:
import numpy as np
import tensorflow as tf

x_data = np.array([[0, 0], [0, 1], [1, 0], [1, 1]], dtype=np.float32)
y_data = np.array([[0], [1], [1], [0]], dtype=np.float32)

X = tf.placeholder(tf.float32)
Y = tf.placeholder(tf.float32)

W1 = tf.Variable(tf.random_normal([2,5]), name="wieght1")
W2 = tf.Variable(tf.random_normal([5,5]), name="wieght2")
W3 = tf.Variable(tf.random_normal([5,5]), name="wieght3")
W4 = tf.Variable(tf.random_normal([5,5]), name="wieght4")
W5 = tf.Variable(tf.random_normal([5,5]), name="wieght5")
W6 = tf.Variable(tf.random_normal([5,5]), name="wieght6")
W7 = tf.Variable(tf.random_normal([5,5]), name="wieght7")
W8 = tf.Variable(tf.random_normal([5,5]), name="wieght8")
W9 = tf.Variable(tf.random_normal([5,5]), name="wieght9")
W10 = tf.Variable(tf.random_normal([5,5]), name="wieght10")
W11 = tf.Variable(tf.random_normal([5,1]), name="wieght11")

b1 = tf.Variable(tf.random_normal([5]), name="bias1")
b2 = tf.Variable(tf.random_normal([5]), name="bias2")
b3 = tf.Variable(tf.random_normal([5]), name="bias3")
b4 = tf.Variable(tf.random_normal([5]), name="bias4")
b5 = tf.Variable(tf.random_normal([5]), name="bias5")
b6 = tf.Variable(tf.random_normal([5]), name="bias6")
b7 = tf.Variable(tf.random_normal([5]), name="bias7")
b8 = tf.Variable(tf.random_normal([5]), name="bias8")
b9 = tf.Variable(tf.random_normal([5]), name="bias9")
b10 = tf.Variable(tf.random_normal([5]), name="bias10")
b11 = tf.Variable(tf.random_normal([1]), name="bias11")


L1 = tf.nn.relu(tf.matmul(X, W1) + b1)
L2 = tf.nn.relu(tf.matmul(L1, W2) + b2)
L3 = tf.nn.relu(tf.matmul(L2, W3) + b3)
L4 = tf.nn.relu(tf.matmul(L3, W4) + b4)
L5 = tf.nn.relu(tf.matmul(L4, W5) + b5)
L6 = tf.nn.relu(tf.matmul(L5, W6) + b6)
L7 = tf.nn.relu(tf.matmul(L6, W7) + b7)
L8 = tf.nn.relu(tf.matmul(L7, W8) + b8)
L9 = tf.nn.relu(tf.matmul(L8, W9) + b9)
L10 = tf.nn.relu(tf.matmul(L9, W10) + b10)

hypothesis = tf.sigmoid(tf.matmul(L10, W11) + b11)

# Weight가 발산하는 문제가 발생해 learning_rate 조절 (더 작게 설정)
cost = -tf.reduce_mean(Y * tf.log(hypothesis) + (1 - Y) * tf.log(1 - hypothesis))
train = tf.train.GradientDescentOptimizer(learning_rate=0.0005).minimize(cost)

predicted = tf.cast(hypothesis > 0.5, dtype=tf.float32)
accuracy = tf.reduce_mean(tf.cast(tf.equal(predicted, Y), dtype=tf.float32))

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    for step in range(30001):
        sess.run(train, feed_dict={X: x_data, Y: y_data})
#         if step % 100 == 0:
#           print(step, sess.run(cost, feed_dict={X: x_data, Y: y_data}), sess.run(W1))
            
    h, c, a = sess.run([hypothesis, predicted, accuracy], feed_dict={X: x_data, Y: y_data})
    print("\nHypothesis: ", h, "\nCorrect: ", c, "\nAccuracy: ", a)


Hypothesis:  [[0.0062361 ]
 [0.9734866 ]
 [0.97346425]
 [0.0062361 ]] 
Correct:  [[0.]
 [1.]
 [1.]
 [0.]] 
Accuracy:  1.0


위에서 진행했던 neural network에서 hidden layer의 activation function을 모두 ReLU로 바꿨습니다. 정확도 100%를 기록했네요. (weight값이 발산하는 현상이 생겨 hyper parameter를 바꿨습니다.)